In [1]:
# executeme

using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
# executeme

using QuantumOptics

# https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
# using TensorOperations

using BenchmarkTools

using LinearAlgebra

# Initial Parameters

In [3]:
Nx = 8; Ny = 8; N=Nx*Ny; q = Nx; cut_off = q
PN = [0,1,2,3,4]
U = 5;
Ncut = cut_off;

# Essentials Functions for Single-Particle

In [4]:
sp_basis = NLevelBasis(N)
sp_matrix = Hofstadter_SP(Nx, Ny, 1/q, 0);

In [5]:
# executeme

function get_sp_op(sp_basis, sp_matrix)
  
    H = SparseOperator(sp_basis)

    for m in 1:N
        for n in 1:N
            H += sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op (generic function with 1 method)

In [6]:
H1 = get_sp_op(sp_basis, sp_matrix);

In [7]:
#check operator form
eigenenergies(dense(H1)) == eigvals(sp_matrix)

true

In [8]:
# executeme

function get_sub_states(sp_op, cut_off)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[1:cut_off]
    
    return states
end

get_sub_states (generic function with 1 method)

In [9]:
sub_states = get_sub_states(H1, cut_off);

In [10]:
# executeme

function get_projector_op(states, basis)
    
    b_sub = SubspaceBasis(basis,states)
    P = projector(b_sub, basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_projector_op (generic function with 1 method)

In [11]:
b_sub, P, Pt = get_projector_op(sub_states, sp_basis);

In [12]:
# executeme

function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end

get_subspace_op (generic function with 1 method)

In [13]:
H1_sub = get_subspace_op(H1, P, Pt);

In [14]:
# executeme

function get_num_sub_list(sp_basis, P, Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(sp_basis, m, m)
        NMP = get_subspace_op(NM, P, Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list (generic function with 1 method)

In [15]:
num_sub_list = get_num_sub_list(sp_basis,P,Pt);

# Essentials Functions for Many-Body

In [20]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=64), states:8), states:495)

In [21]:
# executeme

function get_mb_op(basis_mb, basis, sp_op)
    
    Op_MB = SparseOperator(basis_mb)
    
    for i in 1:length(basis)
        for j in 1:length(basis)
            Op_MB += sp_op.data[i,j] * transition(basis_mb, i, j)
        end
    end
    
    return Op_MB
end

get_mb_op (generic function with 1 method)

In [22]:
H1_MB = get_mb_op(basis_mb, b_sub, H1_sub);
# or
# H1_mb = manybodyoperator(basis_mb, H1_sub);

In [20]:
# executeme

function get_num_mb_list(basis_mb, basis, num_sub_list)
    
    num_mb_list = []
    
    for m in 1:N
        NMP = get_mb_op(basis_mb, basis, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    
    return num_mb_list
end

get_num_mb_list (generic function with 1 method)

In [21]:
num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

# Functions for Interaction Terms

In [22]:
function get_hubbard_int(num_op_list, basis_mb, U)

    IT = SparseOperator(basis_mb)
    
    for m in 1:N
        IT += U/2 * ( num_op_list[m] * num_op_list[m] - num_op_list[m] )
    end
    
    return IT
end

get_hubbard_int (generic function with 1 method)

In [23]:
H_Int_MB = get_hubbard_int(num_mb_list, basis_mb, U);

Operator(dim=21x21)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=25), states:5), states:21)
 0.0+0.0im       0.0+0.0im      …          0.0+0.0im
 0.0+0.0im      -0.8+0.0im                 0.0+0.0im
 0.0+0.0im   2.0e-17+1.0e-17im             0.0+0.0im
 0.0+0.0im  -2.0e-17-2.0e-17im             0.0+0.0im
 0.0+0.0im  -2.0e-17+1.0e-17im             0.0+0.0im
 0.0+0.0im   1.0e-17-2.0e-17im  …          0.0+0.0im
 0.0+0.0im       0.0+0.0im         -0.00765132-0.00998513im
 0.0+0.0im       0.0+0.0im            0.010804+7.85542e-5im
 0.0+0.0im       0.0+0.0im         -0.00598331+0.0251554im
 0.0+0.0im       0.0+0.0im         -0.00737808+0.0132231im
 0.0+0.0im       0.0+0.0im      …   0.00714496+0.0067303im
 0.0+0.0im       0.0+0.0im           0.0148699-0.0296539im
 0.0+0.0im       0.0+0.0im          0.00361681+0.0258888im
 0.0+0.0im       0.0+0.0im          0.00974423+0.0504251im
 0.0+0.0im       0.0+0.0im          -0.0178846+0.0188758im
 0.0+0.0im       0.0+0.0im      …    0.012

In [24]:
function get_hubbard_int2(basis, P, Pt)
    
    #@btime begin
    @time begin
    basis2 = basis ⊗ basis

    # interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i

    Vint = SparseOperator(basis2)

    for n in 1:N
        Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
    end

    Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

    Vint_mb = manybodyoperator(basis_mb, Vint_sub)
    end
    
    return Vint_mb
end

get_hubbard_int2 (generic function with 1 method)

In [25]:
H_Int_MB2 = get_hubbard_int2(sp_basis, P, Pt);

  0.471054 seconds (1.48 M allocations: 113.017 MiB, 8.22% gc time, 74.26% compilation time)


In [26]:
# executeme

function get_hubbard_int3(P, Pt, b_sub, cut_off)
    @time begin
   
    P1 = P.data
    P1t = Pt.data

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, cut_off^2, cut_off^2)

    Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    
    Vint_bsub2_mb = manybodyoperator(basis_mb, Vint_bsub2)
    end
    
    return Vint_bsub2_mb
end

get_hubbard_int3 (generic function with 1 method)

In [27]:
H_Int_MB3 = get_hubbard_int3(P, Pt, b_sub, cut_off);

  0.440823 seconds (1.45 M allocations: 94.906 MiB, 7.00% gc time, 80.09% compilation time)


In [28]:
function get_hubbard_int4(P, Pt, b_sub, cut_off)
    
    bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off, PN)

    @time begin
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2cut = bcut ⊗ bcut

    P4re = reshape(P4, cut_off^2, cut_off^2)

    Vint_bsub2 = SparseOperator(b2cut, U/2*P4re)
        
    Vint_mb_cut = manybodyoperator(bcut_mb, Vint_bsub2)
    end
    
    return Vint_mb_cut
end

get_hubbard_int4 (generic function with 1 method)

In [16]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [30]:
H_Int_MB4 = get_hubbard_int4(P, Pt, b_sub, cut_off);

  0.214436 seconds (734.97 k allocations: 59.874 MiB, 15.06% gc time, 63.03% compilation time)


In [17]:
function get_hubbard_int5(P, Pt, b_sub, cut_off)
    
    bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off, PN)
    
    @time begin
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    Vint_mb_cut = SparseOperator(bcut_mb)
        
    for k in 1:Ncut
        for l in 1:Ncut
            for m in 1:Ncut
                for n in 1:Ncut
                    a1t = create(bcut_mb, k)
                    a2t = create(bcut_mb, l)
                    a2  = destroy(bcut_mb, m)      
                    a1  = destroy(bcut_mb, n)      
                    Vint_mb_cut += U/2*P4[k,l,m,n]*a1t*a2t*a2*a1
                end
            end
        end
    end
    end
    
    return Vint_mb_cut
end

get_hubbard_int5 (generic function with 1 method)

In [18]:
H_Int_MB5 = get_hubbard_int5(P, Pt, b_sub, cut_off);

  8.988675 seconds (701.19 k allocations: 3.376 GiB, 3.84% gc time, 3.39% compilation time)


# Controlling Many-Body Function in QoJulia Freamwork

In [33]:
function manybodyoperator_2_Hubbard(basis::ManyBodyBasis, op::SparseOpType)
    N = length(basis)
    S = length(basis.onebodybasis)
    result = SparseOperator(basis)
    occupations = basis.occupations
    rows = QuantumOpticsBase.rowvals(op.data)
    values = QuantumOpticsBase.nonzeros(op.data)
    @inbounds for column=1:S^2, j in QuantumOpticsBase.nzrange(op.data, column)
        row = rows[j]
        value = values[j]
        index = Tuple(CartesianIndices((S, S, S, S))[(column-1)*S^2 + row])
        for m=1:N, n=1:N
            # println("row:", row, " column:"column, ind_left)
            C = QuantumOpticsBase.coefficient(occupations[m], occupations[n], index[1:2], index[3:4])
            if C!=0.
                result.data[m,n] += C*value
            end
        end
    end
    return result
end

manybodyoperator_2_Hubbard (generic function with 1 method)

# Checking Interaction Terms

In [34]:
isapprox( H_Int_MB2, H_Int_MB3 )

true

In [35]:
isapprox( H_Int_MB4, H_Int_MB5 )

true

In [36]:
isapprox( H_Int_MB4.data, H_Int_MB5.data )

true

In [37]:
isapprox( H_Int_MB3.data , H_Int_MB4.data )

true

# Constructing Total Hamiltonians 

In [38]:
H_MB = H1_MB + H_Int_MB
H_MB2 = H1_MB + H_Int_MB2
H_MB3 = H1_MB + H_Int_MB3
H_MB4 = H1_MB.data + H_Int_MB4.data;

In [23]:
bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off,PN)
H1cut = SparseOperator(bcut_mb)
H1cut.data = H1_MB.data
H1_MB

H_MB5 = H1cut + H_Int_MB5;

# Exact Diagonalization

In [42]:
E = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E0 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

E2 = eigenenergies(dense((H_MB2+dagger(H_MB2))/2))

E3 = eigenenergies(dense((H_MB3+dagger(H_MB3))/2))

E5 = eigenenergies(dense((H_MB5+dagger(H_MB5))/2))

print(E5[1:cut_off], "\n", 
      E2[1:cut_off], "\n", 
      E3[1:cut_off], "\n",
      E0[1:cut_off])

[-5.930026862496533, -5.930026862496531, -5.930026862496529, -5.930026862496529, -5.9300268624965256]
[-5.930026862496533, -5.93002686249653, -5.930026862496529, -5.930026862496528, -5.9300268624965256]
[-5.93002686249653, -5.930026862496529, -5.930026862496528, -5.930026862496527, -5.9300268624965256]
[-5.9308125874564865, -5.930812587456484, -5.930812587456456, -5.930812587456449, -5.930812587456432]

# Fastest Interaction Function

In [26]:
using DataFrames

E5_states = eigenstates(dense(dense((H_MB5+dagger(H_MB5))/2)))

function get_energies(pn, states, basis)
    PN_Energies = Array{Float64}(undef, length(states[1]), 2)
    for i in 1:length(states[1])
        PN_Energies[i] = round(expect(number(basis), states[2][i])) #expected values (first column)
        PN_Energies[i,2] = states[1][i] #eigen-values (second column)
    end
    df = DataFrame(PN_Energies, :auto)
    df = filter(row -> (row.x1 == pn),  df)
    
    return df
end

get_energies (generic function with 1 method)

In [27]:
get_energies(2.0, E5_states, bcut_mb)

Row,x1,x2
,Float64,Float64
1,2.0,-6.5813
2,2.0,-6.5813
3,2.0,-6.5813
4,2.0,-6.5813
5,2.0,-6.58122
6,2.0,-6.58122
7,2.0,-6.58122
8,2.0,-6.58122
9,2.0,-6.58086
